In [3]:
import pandas as pd
import numpy as np
import random
import os
import requests
from time import sleep
from bs4 import BeautifulSoup
from tqdm import tqdm

%load_ext autoreload
%autoreload 2

#### Collect the authorid based on the professors' names  in CUHKSZ SDS website and the final form:

|authorId|name|
|---|---|
|$ID_1$|San, Zhang|
|$ID_2$|Si, Li|
|...|...|
|$ID_n$|Wu, Wang|

In [ ]:
raw_scholar_info = pd.read_csv("Raw_Scholar_Info.csv")
raw_scholar_info = raw_scholar_info.values

#### Use semantic scholar's api to collect the papers' information based on professor's id

|paperId|title|abstract|fieldsOfStudy|authors|
|---|---|---|---|---|
|$ID_1$|title1|abstract1|Mathematics|{'authorId': '$AuthorID_1$', 'name':Zhang San'}|
|$ID_2$|title2|abstract2|Economics|{'authorId': '$AuthorID_2$', 'name':LI Si'}|
|...|...|...|...|...|
|$ID_n$|titlen|abstractn|Computer Science|{'$AuthorID_n$': 'authoridn', 'name':'WANG Wu'}|



In [ ]:
max_query_size = 30
start_idx = 0
end_idx = max_query_size
api_author_batch = 'https://api.semanticscholar.org/graph/v1/author/batch'
paper_info_list = []

for i in tqdm(range(len(raw_scholar_info)//max_query_size + 1)):
    id_list = list(map(str, raw_scholar_info[start_idx:end_idx, 0]))
    id_json = {"ids": id_list}
    sleep(3 + random.normalvariate(0,1))
    author_paper_collections = requests.post(
        api_author_batch,
        params={"fields": "papers,papers.paperId,papers.title,papers.authors,papers.fieldsOfStudy,papers.abstract"},
        json=id_json
    ).json()

    cnt = 0
    for collection in author_paper_collections:
        authorId = collection["authorId"]
        paper_info_list += collection["papers"]
    start_idx = end_idx
    end_idx = min(len(raw_scholar_info), end_idx+max_query_size)

In [ ]:
paper_info_df = pd.DataFrame.from_dict(paper_info_list)
paper_info_df.to_csv('Raw_Paper_Info.csv', index=None)

#### You can import this csv by clone the code in github and it is saved as 'Raw_Paper_Info.csv' in .../raw/CUHKSZ_AcademicGraph-rawdata_released/

#### -------------------------------------------------------------------------------------------------------------------

#### Use semantic scholar's api to collect the papers' citation and reference list and get the intersection part after dropna()
|paperId|ref_paperId|
|---|---|
|$paperID_1$|$refID_1$|
|$paperID_2$|$refID_2$|
|...|...|
|$paperID_n$|$refID_n$|



In [ ]:
paper_info_a= paper_info_df.values

In [ ]:
max_query_size = 50
start_idx = 0
end_idx = max_query_size
api_paper_batch = 'https://api.semanticscholar.org/graph/v1/paper/batch'
ref_info_a_list = []
session = requests.Session()
for i in tqdm(range(len(paper_info_a)//max_query_size + 1)):
    id_list = list(map(str, paper_info_a[start_idx:end_idx, 0]))
    id_json = {"ids": id_list}
    sleep(10 + random.normalvariate(0,1))
    author_paper_collections_a = session.post(
        api_paper_batch,
        params={'fields': 'references.paperId'},
        json=id_json
    ).json()
    while  (type(author_paper_collections_a)==str) or (type(author_paper_collections_a)==dict and author_paper_collections_a.get("message")):      
        print("Post Failed!", author_paper_collections_a)
        author_paper_collections_a = requests.post(
            api_paper_batch,
            params={'fields': 'references.paperId'},
            json=id_json
        ).json()
    for collection in author_paper_collections_a:
        paperId = collection["paperId"]
        ref_paper_list = collection["references"]
        if len(ref_paper_list)>0:
            ref_paper_list = list(map(lambda item: {"paperId": paperId, "ref_paperId":item["paperId"]}, ref_paper_list))
            ref_info_a_list += ref_paper_list
    start_idx = end_idx
    end_idx = min(len(paper_info_a), end_idx+max_query_size)


In [ ]:
ref_info_df = pd.DataFrame.from_dict(ref_info_a_list)

In [ ]:
ref_info_df_full = ref_info_df.dropna()
ref_info_df_full =ref_info_df_full.reset_index()
ref_info_df_full = ref_info_df_full.drop('index',axis=1)

In [ ]:
a_set = set()
b_set = set()
fact_list = np.zeros(len(ref_info_df_full))
cnt = 0
for paperid in paper_info_df.values[:,0]:
    a_set.add(paperid)
for i, ref_paperId in enumerate(ref_info_df_full.values[:,1]):
#     print(i,ref_paperId)
    if ref_paperId in a_set:
        fact_list[i] = 1
        cnt += 1
        print(i,ref_paperId)
    else:
        fact_list[i] = 0

ref_info_df_full["fact"] = pd.Series(fact_list, index=ref_info_df_full.index)

In [ ]:
ref_info_df_full= ref_info_df_full[ref_info_df_full['fact']==1].drop(['fact'],axis=1)

In [ ]:
ref_info_df_full.to_csv("Raw_Citations.csv", index=None)

#### It is saved as 'Raw_Citations.csv' in .../raw/CUHKSZ_AcademicGraph-rawdata_released/

#### -------------------------------------------------------------------------------------------------------------------

#### Use semantic scholar's api to collect the papers' embedding 

|paperId|0|1|2|...|767|
|---|---|---|---|---|---|
|$ID_1$|-5.576845169067383|-4.431099891662598|-3.3520641326904297|...|-1.6501665115356443|
|$ID_2$|-0.6059739589691162|-1.057750582695007|2.3096306324005127|...|-0.5611065626144409|
|...|...|...|...|...|...|
|$ID_n$|1.5763059854507446|-0.5848586559295654|2.6198506355285645|...|1.0170499086380005|


In [ ]:
paperId_intersection.to_csv('Raw_Citations.csv')
paperId_intersection = paperId_intersection.values

In [ ]:
paperid_set=set()
for paperid in prof_paper_info.values[:,0]:
    if paperid  in paperid_set:
        1
    else:
    ##move the two papers which do not have embedding
        if paperid =='0ef762d981a57a74666833b92c57d394bc458631' or paperid =='7480bd5c9c538019b53f1a9061e37a5f9be06345':
            1
        else:
            paperid_set.add(paperid)

In [ ]:
paperid_unique = list(paperid_set)
# len(paperid_unique)

In [ ]:
max_query_size = 50
start_idx = 400
end_idx = max_query_size
api_paper_batch = 'https://api.semanticscholar.org/graph/v1/paper/batch'
em_paper_a_list = []
session = requests.Session()

for i in tqdm(range(len(paperid_unique)//max_query_size + 1)):
    id_list = list(paperid_unique[start_idx:end_idx])
    id_json = {"ids": id_list}
    sleep(3 + random.normalvariate(0,0.5))
    em_paper_a = session.post(
        api_paper_batch,
        params={"fields": "embedding"},
        json=id_json
    ).json()
    # if start_idx >=41:
    #     break
    while  (type(em_paper_a)==str) or (type(em_paper_a)==dict and em_paper_a.get("message")):      
        print("Post Failed!", em_paper_a)
        em_paper_a = session.post(
            api_paper_batch,
            params={"fields": "embedding"},
            json=id_json
        ).json()
    for collection in em_paper_a:
        # print(collection)
        paperid = collection['paperId']
        if collection['embedding'] == None:
            print('1')
        else:
            vec = collection['embedding']['vector']
            em_paper_info_dict = {"paperId": paperid}

            for i, value in enumerate(vec):
                em_paper_info_dict[str(i)] = value
            em_paper_a_list.append(em_paper_info_dict)
        
    
    # break
    
    start_idx = end_idx
    end_idx = min(len(paperid_unique), end_idx+max_query_size)

In [ ]:
em_df = pd.DataFrame.from_dict(em_paper_a_list)
em_df.to_csv('Raw_Paper_Embedding.csv',index=False)

#### It is saved as 'Raw_Paper_Embedding.csv' in .../raw/CUHKSZ_AcademicGraph-rawdata_released/